##FD schemes convergece investigation using 2D wave equation

In [1]:
%load_ext autoreload
%autoreload 2
from sympy import symbols, init_printing, simplify, solve, srepr, Add, Symbol, Integer, Float, pi, cos, sin
from mako.template import Template
from mako.lookup import TemplateLookup
from mako.runtime import Context
from StringIO import StringIO
from fdlib import *
init_printing()

In [2]:
U,V,Txx,Tyy,Txy = IndexedBases('U V Txx Tyy Txy')
b, lam, mu = symbols('beta lambda mu')
x,y,t,h,dt,a,b, Vs = symbols('x y t h dt a b Vs')

In [3]:
accuracy_time = 1 # O(dt^2) accuracy in time domain
accuracy_space = 2 # O(h^4) accuracy in space domain

###Eigenmode functions (as initial condition and analytical solution)

$U = acos(\pi x)sin(\pi y)cos(at)$<br>
$V = -asin(\pi x)cos(\pi y)cos(at)$<br>
$T_{xx} = -bsin(\pi x)sin(\pi y)sin(at)$<br>
$T_{yy} = bsin(\pi x)sin(\pi y)sin(at)$<br>
$T_{xy} = T_{yx} = 0$<br>
where $a=\sqrt 2 \pi V_s\space \space b=2\pi\mu$

In [4]:
a = 2**(0.5)*pi*Vs
b = 2*pi*mu
U_sol = a*cos(pi*x)*sin(pi*y)*cos(a*t)
V_sol = -a*sin(pi*x)*cos(pi*y)*cos(a*t)
Txx_sol = -b*sin(pi*x)*sin(pi*y)*sin(a*t)
Tyy_sol = b*sin(pi*x)*sin(pi*y)*sin(a*t)
Txy_sol = Float(0)

###Initial conditions

In [5]:
# substitution t=0
U_init = print_myccode(U_sol.subs(t,0))
V_init = print_myccode(V_sol.subs(t,0))
Txx_init = print_myccode(Txx_sol.subs(t,0))
Tyy_init = print_myccode(Tyy_sol.subs(t,0))
Txy_init = print_myccode(Txy_sol.subs(t,0))

###Calculate FD approximation of derivatives

In [6]:
l = [t,x,y]

# first-order time derivatives in velocities and stresses
# O(dt^2) accuracy
DtU = Deriv_half(U,l,0,dt,accuracy_time)[1]
DtV = Deriv_half(V,l,0,dt,accuracy_time)[1]

DtTxx = Deriv_half(Txx,l,0,dt,accuracy_time)[1]
DtTxy = Deriv_half(Txy,l,0,dt,accuracy_time)[1]
DtTyy = Deriv_half(Tyy,l,0,dt,accuracy_time)[1]

# first-order spacial derivatives of velocities and stresses
# O(h^4) accuracy

DxU = Deriv_half(U,l,1,h,accuracy_space,shift_forward=False)[1]
DyV = Deriv_half(V,l,2,h,accuracy_space,shift_forward=False)[1]

DyU = Deriv_half(U,l,2,h,accuracy_space)[1]
DxV = Deriv_half(V,l,1,h,accuracy_space)[1]

DxTxx = Deriv_half(Txx,l,1,h,accuracy_space)[1]
DyTyy = Deriv_half(Tyy,l,2,h,accuracy_space)[1]

DxTxy = Deriv_half(Txy,l,1,h,accuracy_space,shift_forward=False)[1]
DyTxy = Deriv_half(Txy,l,2,h,accuracy_space,shift_forward=False)[1]

In [7]:
DxTxx

In [8]:
# note difference between shift_forward True/False
DxTxy

In [9]:
DtTxx

###PDEs

#####momentum equations
$\partial_tU = b(\partial_xT_{xx}+\partial_yT_{xy})$<br>
$\partial_tV = b(\partial_xT_{xy}+\partial_yT_{yy})$<br>
#####stress-strain equations
$\partial_tT_{xx} = (\lambda+2\mu)\partial_xU + \lambda\partial_yV $<br>
$\partial_tT_{yy} = (\lambda+2\mu)\partial_yV + \lambda\partial_xU $<br>
$\partial_tT_{xy} = \mu(\partial_yU + \partial_xV)$<br>

In [10]:
# momentum equations
eq1 = Eq(DtU, b*(DxTxx + DyTxy))
eq2 = Eq(DtV, b*(DxTxy + DyTyy))

# stress-strain equations
eq3 = Eq(DtTxx, (lam + 2*mu)*DxU + lam*DyV)
eq4 = Eq(DtTyy, (lam + 2*mu)*DyV + lam*DxU)
eq5 = Eq(DtTxy, mu*(DyU+DxV))

In [11]:
eq1

In [15]:
# for demo only
simplify(solve(eq1,U[t+1,x,y])[0]-U[t,x,y])+U[t,x,y]

In [16]:
# for demo only
simplify(solve(eq3,Txx[t+1,x,y])[0]-Txx[t,x,y])+Txx[t,x,y]

In [17]:
# for demo only
# note difference between Txx and Txy indices
simplify(solve(eq5,Txy[t+1,x,y])[0]-Txy[t,x,y])+Txy[t,x,y]

###solve for FD stencil

In [18]:
# generate computation code segments
U_t = print_assignment(eq1, U[t+1,x,y], U[t,x,y], pochoir=True)
V_t = print_assignment(eq2, V[t+1,x,y], V[t,x,y], pochoir=True)
Txx_t = print_assignment(eq3, Txx[t+1,x,y], Txx[t,x,y], pochoir=True)
Tyy_t = print_assignment(eq4, Tyy[t+1,x,y], Tyy[t,x,y], pochoir=True)
Txy_t = print_assignment(eq5, Txy[t+1,x,y], Txy[t,x,y], pochoir=True)

In [19]:
# they look like this
Txx_t

'Txx(t + 1,x,y)=(1.0/24.0)*dt*(27*lambda*U(t,x,y) + lambda*U(t,x - 2,y) - 27*lambda*U(t,x - 1,y) - lambda*U(t,x + 1,y) + 27*lambda*V(t,x,y) + lambda*V(t,x,y - 2) - 27*lambda*V(t,x,y - 1) - lambda*V(t,x,y + 1) + 54*mu*U(t,x,y) + 2*mu*U(t,x - 2,y) - 54*mu*U(t,x - 1,y) - 2*mu*U(t,x + 1,y))/h + Txx(t,x,y)'

In [20]:
# and this
U_t

'U(t + 1,x,y)=(1.0/12.0)*M_PI*dt*mu*(-27*Txx(t,x,y) + Txx(t,x - 1,y) + 27*Txx(t,x + 1,y) - Txx(t,x + 2,y) + 27*Txy(t,x,y) + Txy(t,x,y - 2) - 27*Txy(t,x,y - 1) - Txy(t,x,y + 1))/h + U(t,x,y)'

###boundary conditions

In [ ]:
# constant boundary condition
bc_const = Float(0)
# Dirichlet boundary condition
bc_dirichlet = 100 + 0.2*t
# Neumann boundary condition (expression evaluates to zero)
bc_neumann = Deriv(U,l,1,h,2)[1] - 1

In [ ]:
bc_neuman1 = solve(bc_neumann,U[t,x+1,y,z])[0].subs(x+1,x); assign_pochoir_bc(bc_neuman1)

In [ ]:
bc_U_x0 = assign_pochoir_bc(bc_const); bc_U_x1 = assign_pochoir_bc(bc_const)
bc_U_y0 = assign_pochoir_bc(bc_const); bc_U_y1 = assign_pochoir_bc(bc_const)
bc_U_z0 = assign_pochoir_bc(bc_const); bc_U_z1 = assign_pochoir_bc(bc_const)

In [ ]:
bc_V_x0 = assign_pochoir_bc(bc_const); bc_V_x1 = assign_pochoir_bc(bc_const)
bc_V_y0 = assign_pochoir_bc(bc_const); bc_V_y1 = assign_pochoir_bc(bc_const)
bc_V_z0 = assign_pochoir_bc(bc_const); bc_V_z1 = assign_pochoir_bc(bc_const)

In [ ]:
bc_W_x0 = assign_pochoir_bc(bc_const); bc_W_x1 = assign_pochoir_bc(bc_const)
bc_W_y0 = assign_pochoir_bc(bc_const); bc_W_y1 = assign_pochoir_bc(bc_const)
bc_W_z0 = assign_pochoir_bc(bc_const); bc_W_z1 = assign_pochoir_bc(bc_const)

In [ ]:
bc_Txx_x0 = assign_pochoir_bc(bc_const); bc_Txx_x1 = assign_pochoir_bc(bc_const)
bc_Txx_y0 = assign_pochoir_bc(bc_const); bc_Txx_y1 = assign_pochoir_bc(bc_const)
bc_Txx_z0 = assign_pochoir_bc(bc_const); bc_Txx_z1 = assign_pochoir_bc(bc_const)

In [ ]:
bc_Tyy_x0 = assign_pochoir_bc(bc_const); bc_Tyy_x1 = assign_pochoir_bc(bc_const)
bc_Tyy_y0 = assign_pochoir_bc(bc_const); bc_Tyy_y1 = assign_pochoir_bc(bc_const)
bc_Tyy_z0 = assign_pochoir_bc(bc_const); bc_Tyy_z1 = assign_pochoir_bc(bc_const)

In [ ]:
bc_Tzz_x0 = assign_pochoir_bc(bc_const); bc_Tzz_x1 = assign_pochoir_bc(bc_const)
bc_Tzz_y0 = assign_pochoir_bc(bc_const); bc_Tzz_y1 = assign_pochoir_bc(bc_const)
bc_Tzz_z0 = assign_pochoir_bc(bc_const); bc_Tzz_z1 = assign_pochoir_bc(bc_const)

In [ ]:
bc_Txy_x0 = assign_pochoir_bc(bc_const); bc_Txy_x1 = assign_pochoir_bc(bc_const)
bc_Txy_y0 = assign_pochoir_bc(bc_const); bc_Txy_y1 = assign_pochoir_bc(bc_const)
bc_Txy_z0 = assign_pochoir_bc(bc_const); bc_Txy_z1 = assign_pochoir_bc(bc_const)

In [ ]:
bc_Tyz_x0 = assign_pochoir_bc(bc_const); bc_Tyz_x1 = assign_pochoir_bc(bc_const)
bc_Tyz_y0 = assign_pochoir_bc(bc_const); bc_Tyz_y1 = assign_pochoir_bc(bc_const)
bc_Tyz_z0 = assign_pochoir_bc(bc_const); bc_Tyz_z1 = assign_pochoir_bc(bc_const)

In [ ]:
bc_Txz_x0 = assign_pochoir_bc(bc_const); bc_Txz_x1 = assign_pochoir_bc(bc_const)
bc_Txz_y0 = assign_pochoir_bc(bc_const); bc_Txz_y1 = assign_pochoir_bc(bc_const)
bc_Txz_z0 = assign_pochoir_bc(bc_const); bc_Txz_z1 = assign_pochoir_bc(bc_const)

###output using templates

In [ ]:
# write to template file
mylookup = TemplateLookup(directories=['templates/'])
mytemplate = mylookup.get_template('test_pochoir_iso_elastic_tmpl.cpp')
buf = StringIO()
dict1 = {'time_period':2,'Txx':Txx_t,'Tyy':Tyy_t,'Tzz':Tzz_t,'Txy':Txy_t,'Txz':Txz_t,'Tyz':Tyz_t,'U':U_t,'V':V_t,'W':W_t}
dict1.update({'bc_U_x0':bc_U_x0,'bc_U_x1':bc_U_x1,'bc_U_y0':bc_U_y0,'bc_U_y1':bc_U_y1,'bc_U_z0':bc_U_z0,'bc_U_z1':bc_U_z1})
dict1.update({'bc_V_x0':bc_V_x0,'bc_V_x1':bc_V_x1,'bc_V_y0':bc_V_y0,'bc_V_y1':bc_V_y1,'bc_V_z0':bc_V_z0,'bc_V_z1':bc_V_z1})
dict1.update({'bc_W_x0':bc_W_x0,'bc_W_x1':bc_W_x1,'bc_W_y0':bc_W_y0,'bc_W_y1':bc_U_y1,'bc_W_z0':bc_W_z0,'bc_W_z1':bc_W_z1})
dict1.update({'bc_Txx_x0':bc_Txx_x0,'bc_Txx_x1':bc_Txx_x1,'bc_Txx_y0':bc_Txx_y0,'bc_Txx_y1':bc_Txx_y1,'bc_Txx_z0':bc_Txx_z0,'bc_Txx_z1':bc_Txx_z1})
dict1.update({'bc_Tyy_x0':bc_Tyy_x0,'bc_Tyy_x1':bc_Tyy_x1,'bc_Tyy_y0':bc_Tyy_y0,'bc_Tyy_y1':bc_Tyy_y1,'bc_Tyy_z0':bc_Tyy_z0,'bc_Tyy_z1':bc_U_z1})
dict1.update({'bc_Tzz_x0':bc_Tzz_x0,'bc_Tzz_x1':bc_Tzz_x1,'bc_Tzz_y0':bc_Tzz_y0,'bc_Tzz_y1':bc_Tzz_y1,'bc_Tzz_z0':bc_Tzz_z0,'bc_Tzz_z1':bc_Tzz_z1})
dict1.update({'bc_Txy_x0':bc_Txy_x0,'bc_Txy_x1':bc_Txy_x1,'bc_Txy_y0':bc_Txy_y0,'bc_Txy_y1':bc_Txy_y1,'bc_Txy_z0':bc_Txy_z0,'bc_Txy_z1':bc_Txy_z1})
dict1.update({'bc_Txz_x0':bc_Txz_x0,'bc_Txz_x1':bc_Txz_x1,'bc_Txz_y0':bc_Txz_y0,'bc_Txz_y1':bc_Txz_y1,'bc_Txz_z0':bc_Txz_z0,'bc_Txz_z1':bc_Txz_z1})
dict1.update({'bc_Tyz_x0':bc_Tyz_x0,'bc_Tyz_x1':bc_Tyz_x1,'bc_Tyz_y0':bc_Tyz_y0,'bc_Tyz_y1':bc_Tyz_y1,'bc_Tyz_z0':bc_Tyz_z0,'bc_Tyz_z1':bc_Tyz_z1})
ctx = Context(buf, **dict1)
mytemplate.render_context(ctx)
code = buf.getvalue()

In [ ]:
# generate compilable C++ source code
f= open('../tests/src/mytest_pochoir.cpp','w')
f.write(code)
f.close()